In [ ]:
import warnings
import time
warnings.filterwarnings(action='ignore')
import re
from tqdm import tqdm

service = re.compile("sevice[0-9]{2} on")
cu_url = "https://cu.bgfretail.com/store/list.do?category=store"
cugugun_url = "https://cu.bgfretail.com/store/GugunList.do"
cudong_url = "https://cu.bgfretail.com/store/DongList.do"
target_url = "https://cu.bgfretail.com/store/list_Ajax.do"

payload = {"pageIndex" : "1",
"listType" : "",
"jumpoCode" : "",
"jumpoLotto" : "",
"jumpoToto" : "",
"jumpoCash" : "",
"jumpoHour" : "",
"jumpoCafe" : "",
"jumpoDelivery" : "",
"jumpoBakery" : "",
"jumpoFry" : "",
"jumpoMultiDevice" : "",
"jumpoPosCash" : "",
"jumpoBattery" : "",
"jumpoAdderss" : "",
"jumpoSido" : "",
"jumpoGugun" : "",
"jumpodong" : "",
"user_id" : "",
"sido" : "",
"Gugun" : "",
"jumpoName" : "",}
jumpo_name = []
jumpo_address = []
tmp_service=[]
service_on = []
cu_r = requests.post(cu_url,data = payload)
bs = BS(cu_r.text)
sido = [x.text for x in bs.find(id = "sido") if x.text != '\n'][1:] #경기도부터 강원자치도까지
for x in tqdm(sido):
    payload['jumpoSido'] = payload['sido'] = x                      # sido 값을 각 payload에 대입
    cugugun_r = requests.post(cugugun_url,data = payload)           
    gugun = [x['CODE_NAME'] for x in cugugun_r.json()['GugunList']] # 각 지자체에 해당되는 시군을 리스트로 만듦
    for y in tqdm(gugun):
        payload['jumpoGugun'] = payload['Gugun'] = y                # 각 시군 값을 payload의 구 군 값에 대입
        dong_r = requests.post(cudong_url,data = payload)
        dong = [x['CODE_NAME'] for x in dong_r.json()['GugunList']] # 각 시군에 속해 있는 동을 리스트로 만듦
        for z in dong:
            payload["jumpodong"] = z                                # 각 동을 payload의 동에 대입
            for a in range(1,20):
                payload['pageIndex'] = a                            # 각 동마다 있는 다음페이지 pageindex 대입
                target = requests.post(target_url, data=payload)
                target_bs = BS(target.text)
                rt = target_bs.find("table",{'summary' : '매장 상세 정보'}).findAll("tr")
                if pd.read_html(target.text)[0].size != 0 :
                    if rt[1].findAll("td")[0].text.strip() == "등록된 게시물이 없습니다." :
                        break
                    for p in range(1,len(rt)) :
                        for x in rt[p].findAll("li", {'class' : re.compile("sevice[0-9]{2} on")}):
                            tmp_service.append(x.findAll("img")[0]['src'].split("_")[-1].replace(".png", "")) # 서비스 항목들 리스트 추가
                            # service on
                        service_on.append([tmp_service]) # 온전한 리스트에 다시 추가
                        tmp_service = [] # 임시 리스트 초기화
                        jumpo_name.append([rt[p].findAll("td")[0].text.strip()]) # jumpo_name
                        jumpo_address.append([rt[p].findAll("td")[1].find("address").text]) # jumpo_address

        break
    break
a = pd.concat([pd.DataFrame(jumpo_name,columns=['점포명']),pd.DataFrame(jumpo_address,columns=['주소'])], axis=1)
pd.concat([a, pd.DataFrame(service_on,columns=['서비스'])],axis =1)